## Exploring text embeddings, clusterings and look up
This notebook adapts a Pinecone Tutorial
It explores efficine embeddings of text into search graphs, clustering
<br>

*   We use FAISS to efficiently map the text into a graph
*   We then explore different methods to efficiently search the embedding space
*   Voroni Cells
*   Qauntization






In [ ]:
import requests
from io import StringIO
import pandas as pd

In [ ]:
# Extract sample data to embed

res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
data['sentence_A']

0       A group of kids is playing in a yard and an ol...
1       A group of children is playing in the house an...
2       The young boys are playing outdoors and the ma...
3       The kids are playing outdoors near a man with ...
4       The young boys are playing outdoors and the ma...
                              ...                        
4495                      A door is being opened by a man
4496                     Someone is boiling okra in a pot
4497    The man is singing heartily and playing the gu...
4498          A man in blue has a yellow ball in the mitt
4499                 Three dogs are resting on a sidewalk
Name: sentence_A, Length: 4500, dtype: object

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [ ]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [ ]:
data['sentence_A']

0       A group of kids is playing in a yard and an ol...
1       A group of children is playing in the house an...
2       The young boys are playing outdoors and the ma...
3       The kids are playing outdoors near a man with ...
4       The young boys are playing outdoors and the ma...
                              ...                        
4495                      A door is being opened by a man
4496                     Someone is boiling okra in a pot
4497    The man is singing heartily and playing the gu...
4498          A man in blue has a yellow ball in the mitt
4499                 Three dogs are resting on a sidewalk
Name: sentence_A, Length: 4500, dtype: object

In [ ]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())
print(len(set(sentences)))

<ipython-input-11-80bc68367cc3>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-11-80bc68367cc3>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-11-80bc68367cc3>:7: F

14505


<ipython-input-11-80bc68367cc3>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)


In [ ]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]
len(sentences)

14504

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ac0c3f7c96f3f5500cc734fa5962c2a4315702e56deb3feb727c482516fa970e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
# as alternative to below could also use this:
import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"

# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")
    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

In [ ]:
# !apt install libomp-dev  # NOT SURE IF I NEED THIS?
# !python -m pip install --upgrade faiss faiss-gpu
!pip install faiss-cpu
# import faiss
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.4 MB/s eta 0:00:00


In [ ]:
# Vanilla Faiss: this will create an index to do quiz searches
# https://www.pinecone.io/learn/faiss-tutorial/
d = sentence_embeddings.shape[1]
print(d)
index = faiss.IndexFlatL2(d)
index.is_trained

768


True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

14504

In [ ]:
# Then search given a query `xq` and number of nearest neigbors to return `k`
k = 4
xq = model.encode(["Someone sprints with a football"])
%time
D, I = index.search(xq, k)  # search
print(I)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 18.4 µs
[[ 9430  3066 13462   245]]


In [ ]:
import numpy as np
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)
print(vecs.shape)
vecs[0][:100]

(4, 768)


array([ 0.01627017,  0.22325909, -0.15037386, -0.30747277, -0.27122417,
       -0.10593203, -0.06460906,  0.04738198, -0.73349065, -0.37657702,
       -0.76762819,  0.1690291 ,  0.53107673,  0.51176614,  1.14415824,
       -0.08562881, -0.67240089, -0.96637088,  0.02545483, -0.21559834,
       -1.25656641, -0.82982188, -0.09824977, -0.21850878,  0.5061025 ,
        0.10527938,  0.50396883,  0.6524297 , -1.39458668,  0.65847492,
       -0.21525349, -0.22487436,  0.81818324,  0.08464295, -0.76141709,
       -0.2892831 , -0.09825782, -0.7304613 ,  0.07855817, -0.84354568,
       -0.59242076,  0.7747137 , -1.20920575, -0.2275797 , -1.30733621,
       -0.23081495, -1.31322515,  0.01629095, -0.97285444,  0.19308169,
        0.47424543,  1.18920887, -1.96741247, -0.70061117, -0.29638764,
        0.60533708,  0.62407458, -0.70340389, -0.86754179,  0.17673109,
       -0.19170582, -0.02951964,  0.2262352 , -0.16695435, -0.80402535,
       -0.45918941,  0.69675511, -0.24928196, -1.01478696, -0.92

## CREATING A VORONOI CELL

In [ ]:
# previous index used exhaustive search
# this time we DO need to train
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [ ]:
%time
D, I = index.search(xq, k)  # search
print(I)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 19.1 µs
[[9430 3066  245 1240]]


In [ ]:
# We can increase the number of nearby cells to search too with `nprobe`.
index.nprobe = 10

In [ ]:
%time
D, I = index.search(xq, k)  # search
print(I)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
[[ 9430  3066 13462   245]]


In [ ]:
index.make_direct_map()

In [ ]:
index.reconstruct(7460)[:100]

array([ 8.9132860e-02, -6.3396865e-01,  1.9877975e+00,  6.6278398e-01,
        1.5730362e-01, -4.1321868e-01, -5.6363726e-01,  6.6167730e-01,
        1.4500333e-01, -8.6914790e-01, -4.1229287e-01,  1.2263156e+00,
        5.5987714e-03,  4.3361554e-01,  5.7804650e-01, -2.4506076e-01,
       -4.3835178e-01, -7.0624721e-01, -4.0217298e-01, -2.1868351e-01,
       -2.3944259e-01,  5.0018060e-01,  6.9506991e-01, -5.3320134e-01,
        4.2437220e-01, -1.3212541e+00, -7.2468007e-01, -1.0654114e+00,
       -1.5549091e+00,  5.6629669e-02, -5.9773421e-01,  8.3731031e-01,
       -9.3474992e-02, -2.3861866e-01, -5.6279665e-01,  1.0481772e+00,
        1.4282235e+00, -6.3106805e-01,  3.6011609e-01, -4.5607248e-01,
        9.0859783e-01,  3.0296519e-01, -3.7764919e-01,  3.7724501e-01,
       -1.2597157e+00, -1.5402514e-01, -4.1926378e-01, -3.5731718e-01,
       -2.8062913e-01, -1.2934328e+00,  4.6962935e-01, -1.6169506e-01,
        4.0047824e-02,  2.5270431e-04, -4.7331971e-01,  5.8443826e-01,
      

## QAUNTIZER

In [ ]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value
%time
D, I = index.search(xq, k)
print(I)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
[[ 5970  8952 10966 12823]]


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['5970: A football player kicks the ball.',
 '8952: Two teammates are lifting a football player in a red and white uniform',
 '10966: Two football players are scrambling for the ball on the court',
 '12823: A football player in a red and white uniform is being lifted by two teammates']